In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ddos-data/out.csv


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import dask.dataframe as dd
import os
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV

In [2]:
df = pd.read_csv("../input/ddos-data/out.csv")

In [4]:
df.describe()

,Unnamed: 0,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,...,Fwd Seg Size Avg,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Min,Label
count,1.513949e+07,1.513949e+07,1.513949e+07,1.513949e+07,1.513949e+07,1.513949e+07,1.513949e+07,1.513949e+07,1.513949e+07,1.513949e+07,...,1.513949e+07,1.513949e+07,1.513949e+07,1.513949e+07,1.513949e+07,1.513949e+07,1.513949e+07,1.513949e+07,1.513949e+07,1.513949e+07
mean,7.569746e+06,8.920319e+03,8.770264e+00,1.226366e+07,2.358934e+01,6.334113e+00,9.783180e+02,2.019014e+02,1.113722e+01,5.057569e+01,...,5.057569e+01,8.829068e+03,8.740443e+03,1.798149e+01,1.736567e+05,8.675735e+04,2.635969e+05,1.160456e+05,4.690261e+06,1.702228e-01
std,4.370395e+06,1.869813e+04,4.929919e+00,3.096609e+07,1.520511e+03,1.640832e+02,6.301678e+04,3.042774e+02,2.428807e+01,6.063933e+01,...,6.063933e+01,1.626557e+04,2.066946e+04,7.715197e+00,2.510512e+06,1.515901e+06,3.324654e+06,2.118789e+06,1.582677e+07,3.758284e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,-1.000000e+00,-1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.784873e+06,5.300000e+01,6.000000e+00,5.140000e+02,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,-1.000000e+00,-1.000000e+00,8.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,7.569746e+06,8.000000e+01,6.000000e+00,2.269900e+04,2.000000e+00,1.000000e+00,4.300000e+01,4.100000e+01,0.000000e+00,3.600000e+01,...,3.600000e+01,9.800000e+02,-1.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,1.135462e+07,3.389000e+03,1.700000e+01,3.050693e+06,5.000000e+00,4.000000e+00,4.910000e+02,3.120000e+02,3.000000e+01,6.800000e+01,...,6.800000e+01,8.192000e+03,2.370000e+02,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,1.513949e+07,6.553500e+04,1.700000e+01,1.200000e+08,3.096290e+05,1.231180e+05,1.443918e+08,6.444000e+04,1.460000e+03,1.652931e+04,...,1.652931e+04,6.553500e+04,6.553500e+04,5.600000e+01,1.140000e+08,7.523242e+07,1.140000e+08,1.140000e+08,1.200000e+08,1.000000e+00


In [3]:
df.drop(columns='Unnamed: 0',inplace=True)

In [4]:
for column in df:
 if df[column].dtype == 'float64':
    df[column]=pd.to_numeric(df[column], downcast='float')
 if df[column].dtype == 'int64':
    df[column]=pd.to_numeric(df[column], downcast='integer')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15139493 entries, 0 to 15139492
Data columns (total 47 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Dst Port           float32
 1   Protocol           float32
 2   Flow Duration      float32
 3   Tot Fwd Pkts       float32
 4   Tot Bwd Pkts       float32
 5   TotLen Fwd Pkts    float32
 6   Fwd Pkt Len Max    float32
 7   Fwd Pkt Len Min    float32
 8   Fwd Pkt Len Mean   float32
 9   Bwd Pkt Len Max    float32
 10  Bwd Pkt Len Min    float32
 11  Bwd Pkt Len Mean   float32
 12  Flow Byts/s        float32
 13  Flow Pkts/s        float32
 14  Flow IAT Mean      float32
 15  Flow IAT Std       float32
 16  Flow IAT Max       float32
 17  Flow IAT Min       float32
 18  Bwd IAT Tot        float32
 19  Bwd IAT Mean       float32
 20  Bwd IAT Std        float32
 21  Bwd IAT Max        float32
 22  Bwd IAT Min        float32
 23  Fwd PSH Flags      float32
 24  Fwd URG Flags      float32
 25  Fwd Pkts/s      

In [5]:
train_X = df.drop(columns="Label",axis=1)

In [8]:
train_X.head()

,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Bwd Pkt Len Max,...,Down/Up Ratio,Fwd Seg Size Avg,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Min
0,53.0,17.0,2291.0,1.0,1.0,49.0,49.0,49.0,49.000000,164.0,...,1.0,49.000000,-1.0,-1.0,8.0,0.0,0.0,0.0,0.0,0.0
1,80.0,6.0,5785762.0,3.0,1.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,8192.0,8192.0,20.0,0.0,0.0,0.0,0.0,0.0
2,80.0,6.0,13513.0,3.0,4.0,287.0,287.0,0.0,95.666664,935.0,...,1.0,95.666664,65535.0,219.0,20.0,0.0,0.0,0.0,0.0,0.0
3,80.0,6.0,1610.0,3.0,4.0,308.0,308.0,0.0,102.666664,935.0,...,1.0,102.666664,65535.0,219.0,20.0,0.0,0.0,0.0,0.0,0.0
4,53.0,17.0,1115.0,1.0,1.0,36.0,36.0,36.0,36.000000,73.0,...,1.0,36.000000,-1.0,-1.0,8.0,0.0,0.0,0.0,0.0,0.0


In [9]:
train_Y = df['Label']

In [10]:
train_Y.shape

(15139493,)

In [12]:
# for i in X_train.columns:
#     minimum = X_train[i].min()
#     maximum = X_train[i].max()
#     print(minimum)
#     print(maximum)
#     X_train[i] = (X_train[i] - minimum)/(maximum - minimum)
#     X_test[i] = (X_test[i] - minimum)/(maximum - minimum)

In [ ]:
del()

In [11]:

from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.tree import DecisionTreeClassifier

#Create an object of the classifier.
bbc = BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(),
                                sampling_strategy='auto',
                                replacement=False,
                                random_state=0)

In [ ]:
bbc.fit(train_X, train_Y)

In [13]:
del(df)

In [14]:
scaler= MinMaxScaler();
X_train_sc = pd.DataFrame(scaler.fit_transform(train_X), index=train_X.index, columns=train_X.columns)

In [15]:
del(train_X)

In [16]:
# # params = {'C': [ 0.01, 0.1, 10, 100] }
# # x_clf_1 = LogisticRegression(n_jobs=-1)

# model1 = LogisticRegression(C=100, penalty='l2', random_state=42, n_jobs=-1, verbose=1)
# model1.fit(X_train_sc,train_Y)

In [17]:
from sklearn.metrics import f1_score

In [18]:
# ypred = model1.predict(X_train_sc)
# # print(ypred)
# f1score = f1_score(train_Y, np.round(ypred))
# print('f1_score for train:',f1score)

# # ypred = model1.predict(X_test)
# # f1score = f1_score(y_test, np.round(ypred))
# # print('f1_score for test:',f1score)

# # # Dumping model in pickle file
# # with open('LR.pkl', 'wb') as fp:
# #     pickle.dump(model1, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [19]:
params = {'penalty':['l1','l2','elasticnet','none'],'C': [ 0.01, 0.1, 10, 100],'solver':['lbfgs','newton-cg','liblinear','sag','saga'],
          'max_iter':[50,100,1000]
         }
x_clf_1 = LogisticRegression(n_jobs=-1)

random_clf_1 = RandomizedSearchCV(x_clf_1, param_distributions=params, scoring='f1', verbose=4, cv=2)

In [ ]:
random_clf_1.fit(X_train_sc,train_Y)

Fitting 2 folds for each of 10 candidates, totalling 20 fits
[CV 1/2] END C=10, max_iter=1000, penalty=l1, solver=newton-cg;, score=nan total time=   3.1s
[CV 2/2] END C=10, max_iter=1000, penalty=l1, solver=newton-cg;, score=nan total time=   3.1s


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1484: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio parameters"


[CV 1/2] END C=0.1, max_iter=50, penalty=none, solver=lbfgs;, score=0.778 total time=12.8min


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1484: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio parameters"


[CV 2/2] END C=0.1, max_iter=50, penalty=none, solver=lbfgs;, score=0.781 total time=13.1min
[CV 1/2] END C=100, max_iter=1000, penalty=elasticnet, solver=newton-cg;, score=nan total time=   3.8s
[CV 2/2] END C=100, max_iter=1000, penalty=elasticnet, solver=newton-cg;, score=nan total time=   3.2s
[CV 1/2] END C=100, max_iter=1000, penalty=l1, solver=saga;, score=0.787 total time= 3.1min
[CV 2/2] END C=100, max_iter=1000, penalty=l1, solver=saga;, score=0.788 total time= 2.8min


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:1526: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(effective_n_jobs(self.n_jobs))
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the 

In [ ]:
# params = {'C': [ 0.01, 0.1, 10, 100] }
# x_clf_1 = LogisticRegression(n_jobs=-1)

model1 = LogisticRegression(C=100, penalty='l2', random_state=42, n_jobs=-1, verbose=1)
model1.fit(X_train_sc,train_Y)